In [1]:
import numpy as np
import pandas as pd

### Read in Data

In [2]:
# USGS and FEMA hurricane and flood data
hrcn = pd.read_csv('HurricanewithZIP.csv')
floods = pd.read_csv('Floods Listing - Flood IDs.csv')

In [3]:
# FEMA grant data
appls = pd.read_csv('GeoPublicAssistanceApplicants.csv')
funded = pd.read_csv('PublicAssistanceFundedProjectsDetails.csv')

In [4]:
# Census and IRS data
pop = pd.read_csv('pop_totals_by_zip.csv')
taxes = pd.read_csv('basic_taxes_by_zip.csv')

### Functions

In [5]:
def deciler(df, cat_column, dec_column):
    '''adds new column of deciles, PER UNIQUE CATEGORY IN cat_column
    to dataframe from input column (string), names it dec_column_decile_by_cat_column
    returns new dataframe with new column, input dataframe remains unaltered'''
    # copy the dataframe
    dataframe = df.copy()
    # create new column set with dummys
    dataframe[f'{dec_column}_decile_by_{cat_column}'] = '_'
    # get categories to group separate quantiles per category 
    categories = dataframe[cat_column].unique()
    # loop through categories and quantile
    for category in categories:
        # grab dec_column data in each category
        data_to_decile = dataframe.loc[dataframe[cat_column] == category, dec_column]
        # create deciles
        deciles = pd.qcut(data_to_decile, 10, labels=False, duplicates='drop')
        # set the new decile column equal to decile value
        dataframe.loc[dataframe[cat_column] == category, f'{dec_column}_decile_by_{cat_column}'] = deciles
    return dataframe

def scaler(df, cat_column, scale_column):
    '''returns new dataframe with new column, "column_scaled"
    scaled by z-score'''
    from scipy.stats import zscore
    dataframe = df.copy()
    dataframe[f'{scale_column}_scaled_by_{cat_column}'] = '_'
    categories = dataframe[cat_column].unique()
    for category in categories:
        data_to_scale = dataframe.loc[dataframe[cat_column] == category, scale_column]
        scaled = zscore(data_to_scale)
        dataframe.loc[dataframe[cat_column] == category, f'{scale_column}_scaled_by_{cat_column}'] = scaled
    return dataframe

def mean_gregator(df, cat_column, agg_column):
    '''returns new dataframe with new column, "agg_column_mean_by_cat_column"'''
    dataframe = df.copy()
    dataframe[f'{agg_column}_mean_by_{cat_column}'] = '_'
    categories = dataframe[cat_column].unique()
    for category in categories:
        data_to_average = dataframe.loc[dataframe[cat_column] == category, agg_column]
        average = data_to_average.mean()
        dataframe.loc[dataframe[cat_column] == category, f'{agg_column}_mean_by_{cat_column}'] = average
    return dataframe

### Floods

In [6]:
floods.head()

,FEMA Disaster Code,FEMA Event Name,USGS Flood Event Name,Flood id,Timeline_Category,Unnamed: 5,USGS Flood Event Name.1,Flood id.1,Timeline_Category.1
0,1292,North Carolina Hurricane Floyd & Irene,Irene,5,1,NaN,2019 Hurricane Dorian,1.0,3.0
1,1306,Florida Hurricane Irene,Irene,5,1,NaN,Florence Sep 2018,2.0,2.0
2,1490,North Carolina Hurricane Isabel,Isabel September 2003,8,0,NaN,Harvey Aug 2017,3.0,2.0
3,1491,Virginia Hurricane Isabel,Isabel September 2003,8,0,NaN,Hermine,4.0,2.0
4,1492,Maryland Hurricane Isabel,Isabel September 2003,8,0,NaN,Irene,5.0,1.0


In [7]:
# drop last three cols
floods.drop(columns=['Flood id', 'Unnamed: 5', 'USGS Flood Event Name.1', 'Flood id.1', 'Timeline_Category.1'], inplace=True)

In [8]:
floods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   FEMA Disaster Code     86 non-null     int64 
 1   FEMA Event Name        86 non-null     object
 2   USGS Flood Event Name  86 non-null     object
 3   Timeline_Category      86 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.8+ KB


### Hurricanes

In [9]:
# need to have data with flood depth above ground, so lets keep from hrcn just rows from height_above_gnd
hrcn = hrcn[hrcn['height_above_gnd'].notnull()].copy()

In [10]:
# filter out clerical noise columns
hrcn = hrcn[['latitude',
            'longitude',
            'site_latitude',
            'site_longitude',
            'eventName',
            'stateName',
            'countyName',
            'hwm_id',
            'hwm_locationdescription',
            'elev_ft',
            'height_above_gnd',
            'hwm_environment',
            'zip']].copy()

In [11]:
# aggregate heights by hurricane
hrcn = deciler(hrcn, 'eventName', 'height_above_gnd')
hrcn = scaler(hrcn, 'eventName', 'height_above_gnd')
hrcn = deciler(hrcn, 'eventName', 'elev_ft')

C:\Users\leejl\anaconda3\lib\site-packages\scipy\stats\stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [12]:
# aggregate height by zipcode
hrcn = deciler(hrcn, 'zip', 'height_above_gnd')
hrcn = mean_gregator(hrcn, 'zip', 'height_above_gnd')

In [13]:
hrcn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6069 entries, 0 to 7865
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   latitude                              6069 non-null   float64
 1   longitude                             6069 non-null   float64
 2   site_latitude                         6069 non-null   float64
 3   site_longitude                        6069 non-null   float64
 4   eventName                             6069 non-null   object 
 5   stateName                             6069 non-null   object 
 6   countyName                            6069 non-null   object 
 7   hwm_id                                6069 non-null   int64  
 8   hwm_locationdescription               5463 non-null   object 
 9   elev_ft                               5875 non-null   float64
 10  height_above_gnd                      6069 non-null   float64
 11  hwm_environment  

### Applications

In [14]:
# drop unnecessary features
appls.drop(columns=['state', 'hash', 'lastRefresh', 'id'], inplace=True)

In [15]:
appls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19119 entries, 0 to 19118
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   disasterNumber  19119 non-null  int64  
 1   applicantId     19119 non-null  object 
 2   applicantName   19119 non-null  object 
 3   addressLine1    19108 non-null  object 
 4   addressLine2    1155 non-null   object 
 5   city            19116 non-null  object 
 6   zipCode         19119 non-null  object 
 7   latitude        19119 non-null  float64
 8   longitude       19119 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.3+ MB


### Funded Applications

In [16]:
funded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956065 entries, 0 to 956064
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   disasterNumber         956065 non-null  int64  
 1   declarationDate        956065 non-null  object 
 2   incidentType           956065 non-null  object 
 3   pwNumber               956065 non-null  int64  
 4   applicationTitle       948678 non-null  object 
 5   applicantId            956065 non-null  object 
 6   damageCategoryCode     956065 non-null  object 
 7   dcc                    956065 non-null  object 
 8   damageCategory         956065 non-null  object 
 9   projectSize            929152 non-null  object 
 10  county                 929152 non-null  object 
 11  countyCode             929152 non-null  float64
 12  state                  956065 non-null  object 
 13  stateCode              956065 non-null  object 
 14  stateNumberCode        956065 non-nu

In [17]:
# drop unnecessary columns
funded.drop(columns=['incidentType', 
                     'pwNumber', 
                     'applicationTitle',
                     'damageCategoryCode',
                     'dcc',
                     'county',
                     'countyCode',
                     'stateCode',
                     'stateNumberCode',
                     'hash', 'lastRefresh', 'id'], inplace=True)

# filter the funded projects to only those disasters that we have applicants for
funded = funded.loc[funded['disasterNumber'].isin(appls['disasterNumber']), :].copy()

In [18]:
# check for anything negative
funded.describe()

,disasterNumber,projectAmount,federalShareObligated,totalObligated
count,158160.000000,1.581600e+05,1.581600e+05,1.581600e+05
mean,3152.218507,2.620620e+05,2.377928e+05,2.381013e+05
std,1276.667971,6.737716e+06,6.157101e+06,6.157532e+06
min,1292.000000,-1.712000e+08,-1.712000e+08,-1.712000e+08
25%,1609.000000,0.000000e+00,1.721648e+03,1.736787e+03
50%,4021.000000,6.326825e+03,7.258090e+03,7.314680e+03
75%,4091.000000,3.521236e+04,3.268113e+04,3.292378e+04
max,4468.000000,1.091636e+09,9.824720e+08,9.824720e+08


In [19]:
# odd negative rows, seems legit except for the sign, take np.abs()
funded['projectAmount'] = np.abs(funded['projectAmount'])
funded['federalShareObligated'] = np.abs(funded['federalShareObligated'])
funded['totalObligated'] = np.abs(funded['totalObligated'])

# check
funded.describe()

,disasterNumber,projectAmount,federalShareObligated,totalObligated
count,158160.000000,1.581600e+05,1.581600e+05,1.581600e+05
mean,3152.218507,2.933997e+05,2.658347e+05,2.662979e+05
std,1276.667971,6.736424e+06,6.155954e+06,6.156377e+06
min,1292.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,1609.000000,1.000000e+03,2.512500e+03,2.532463e+03
50%,4021.000000,8.540000e+03,9.158540e+03,9.227415e+03
75%,4091.000000,4.345397e+04,3.956747e+04,3.985948e+04
max,4468.000000,1.091636e+09,9.824720e+08,9.824720e+08


### Joining Datasets

In [20]:
# join appls and funded into grants
grants = pd.merge(left=funded, right=appls, how='inner', left_on=['applicantId', 'disasterNumber'], right_on=['applicantId', 'disasterNumber'])

In [21]:
# join grants and flood on disasterNumbers in order to apply deciler and scaler
grants = pd.merge(left=grants, right=floods, how='left', left_on='disasterNumber', right_on='FEMA Disaster Code')
grants.drop(columns='FEMA Disaster Code', inplace=True)

In [22]:
# decile and scale
grants = deciler(grants, 'USGS Flood Event Name', 'projectAmount')
grants = scaler(grants, 'USGS Flood Event Name', 'projectAmount')

grants = deciler(grants, 'USGS Flood Event Name', 'federalShareObligated')
grants = scaler(grants, 'USGS Flood Event Name', 'federalShareObligated')

grants = deciler(grants, 'USGS Flood Event Name', 'totalObligated')
grants = scaler(grants, 'USGS Flood Event Name', 'totalObligated')

In [23]:
# format zipcodes -- drop trailing 4 digits and hyphen
grants['zipCode'] = [i[:5] for i in grants['zipCode']]

### Population and Income

In [24]:
# format zipcodes to strings, add leading zeros where necessary
pop['zipcode'] = [i.zfill(5) for i in pop['zipcode'].astype('str')]

# generate scaled population and deciles
from scipy.stats import zscore

pop['population_decile_by_zip'] = pd.qcut(pop['population'], 10, labels=False, duplicates='drop')
pop['population_scaled_by_zip'] = zscore(pop['population'])

pop.head()

,zipcode,population,population_decile_by_zip,population_scaled_by_zip
0,00601,18570,8,0.664237
1,00602,41520,9,2.332875
2,00603,54689,9,3.290360
3,00606,6615,6,-0.204982
4,00610,29016,8,1.423739


In [25]:
# drop state totals
taxes = taxes.loc[taxes['zipcode'] != 0, :]
taxes.head()

,STATE,zipcode,agi_stub,A00100
6,AL,35004,1,19765.0
7,AL,35004,2,45905.0
8,AL,35004,3,57471.0
9,AL,35004,4,46868.0
10,AL,35004,5,53543.0


In [26]:
# format zipcodes to strings, add leading zeros where necessary
taxes['zipcode'] = [i.zfill(5) for i in taxes['zipcode'].astype('str')]

# aggregate agi per zip
total_agi = taxes.groupby('zipcode').sum()
total_agi.drop(columns='agi_stub', inplace=True)
total_agi.rename(columns={'A00100': 'total_agi_per_zip'}, inplace=True)

total_agi.head()

,total_agi_per_zip
zipcode,
01001,436051.0000
01002,664035.0000
01005,118789.0000
01007,439083.0000
01008,35627.0001


In [27]:
# join population data
grants = pd.merge(left=grants, right=pop, how='inner', left_on='zipCode', right_on='zipcode')

In [28]:
# join tax data
grants = pd.merge(left=grants, right=total_agi, how='inner', left_on='zipCode', right_on=total_agi.index)

In [29]:
# generate agi per capita feature
grants['agi_per_capita_per_zip'] = grants['total_agi_per_zip'] / grants['population']

In [30]:
grants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131673 entries, 0 to 131672
Data columns (total 32 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   disasterNumber                                         131673 non-null  int64  
 1   declarationDate                                        131673 non-null  object 
 2   applicantId                                            131673 non-null  object 
 3   damageCategory                                         131673 non-null  object 
 4   projectSize                                            131673 non-null  object 
 5   state                                                  131673 non-null  object 
 6   projectAmount                                          131673 non-null  float64
 7   federalShareObligated                                  131673 non-null  float64
 8   totalObligated                    

### Save to CSVs

In [31]:
# hrcn.to_csv('depths_and_hurricanes.csv')
# grants.to_csv('grants_money_pop.csv')